In [2]:
# importing necessary modules.
import requests
import json
import pandas as pd
import numpy as np
import config2 as cfg
from sqlalchemy import create_engine

In [3]:
# creating a connection to an aws rds postgres cloud database.
rds_connection_string = f'postgres:{cfg.password}@test-db.cy2enoewwvsi.us-east-2.rds.amazonaws.com:5432/stocks_db'
engine = create_engine(f'postgres://{rds_connection_string}')

In [4]:
# checking out the SandP_500 table.
ticker_symbols = []
sandp_russel = pd.read_sql_query('SELECT * FROM sandp_russell', con=engine)
for symbol in sandp_russel['symbol']:
    ticker_symbols.append(symbol)

In [5]:
# creating empty list.
industry_list = []

# creating empty list.
sector_list = []

# looping through ticker_symbols list.
for ticker in ticker_symbols:
    
    # try except statement to ignore missing data.    
    try:

        # obtaining the json data for sector and industry type from financialmodelingprep.          
        response = requests.get(f'https://financialmodelingprep.com/api/v3/company/profile/{ticker}').json()

        # getting industry data.
        industry_list.append(response['profile']['industry'])
        
        # getting sector data.
        sector_list.append(response['profile']['sector'])
    
    # try except statement to ignore missing data.
    except:
        pass

In [6]:
# creating an empty list.
market_cap_list = []

# looping through ticker_symbols list.
for ticker in ticker_symbols:
        
    # try except statement to ignore missing data.    
    try:
        
        # obtaining the json data for market cap from financialmodelingprep.   
        response2 = requests.get(f'https://financialmodelingprep.com/api/v3/enterprise-value/{ticker}').json()
        
        # calculating average market cap value from 2009/2010.             
        market_cap = (int(response2['enterpriseValues'][-1]['Market Capitalization']) + int(response2['enterpriseValues'][-2]['Market Capitalization']))/2

        # appending market cap value to market cap list.          
        market_cap_list.append(market_cap)
    
    # try except statement to ignore missing data.    
    except:
        pass

In [7]:
# adding market cap to dataframe
sandp_russel['market_cap'] = market_cap_list

In [8]:
# adding industry to dataframe.
sandp_russel['industry'] = industry_list

In [9]:
# adding sector to dataframe.
sandp_russel['sector'] = sector_list

In [10]:
# viewing resuts.
sandp_russel.head()

,symbol,name,price_to_bookvalue,price_to_book,price_to_sales,price_to_earnings,receivables_turnover,price_to_free_cash_flow,price_to_operating_cash_flow,enterprise_value_multiple,...,asset_growth,book_value_per_share_growth,debt_growth,randd_expense_growth,sganda_expense_growth,percent_return_on_investment,above_below_sandp_return,market_cap,industry,sector
0,A,"Agilent Technologies, Inc.",2.5517,5.17165,1.48130,6.30885,7.0014,18.18070,13.57995,16.746223,...,0.18005,0.15185,0.31885,-0.0674,0.01880,215.407785,1,7.462700e+09,Medical Diagnostics & Research,Healthcare
1,AAL,"American Airlines Group, Inc.",-99.0000,-99.00000,0.11565,-99.00000,27.3348,-99.00000,2.20925,61.812855,...,-0.00170,-0.06235,0.00975,-99.0000,0.01435,492.976589,1,6.133809e+08,Airlines,Industrials
2,AAN,"Aaron's, Inc.",1.5315,1.95610,0.78855,12.37860,27.7757,59.12785,19.92670,0.126805,...,0.10410,0.12870,-0.38070,-99.0000,0.08135,153.807704,1,5.597095e+08,Consulting & Outsourcing,Industrials
3,AAON,"AAON, Inc.",3.4176,3.41760,1.63730,16.86505,6.7351,20.69945,10.96230,0.287753,...,0.06795,0.13045,-0.98780,-99.0000,-0.02605,454.948301,1,3.842042e+08,Building Materials,Basic Materials
4,AAP,"Advance Auto Parts, Inc.",2.9566,3.10380,0.70600,14.16840,57.0461,7.41850,5.46130,0.499984,...,0.03670,0.19520,-0.57150,-99.0000,0.10360,158.290852,1,1.912092e+09,Retail - Apparel & Specialty,Consumer Cyclical


In [11]:
# appedending pandas dataframe to cloud postgres database.
sandp_russel.to_sql(name='sandp_russell', con=engine, if_exists="replace", index=False)